<a href="https://colab.research.google.com/github/norojordan/big-data-challenge/blob/main/amazon_watches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [22.8 kB]
Get:14 http://ppa.launch

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-21 02:07:03--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.52MB/s    in 0.2s    

2022-05-21 02:07:03 (5.52 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [15]:
from pyspark import SparkFiles
# Load in Amazon watch reviews from S3 into a DataFrame
url = "https://s3.amazonaws.com/nj-hw-bucket/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

watch_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep=",",inferSchema=True)
watch_df.show(10)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|marketplace	customer_id	review_id	product_id	product_parent	product_title	product_category	star_rating	helpful_votes	total_votes	vine	verified_purchase	review_headline	review_body	review_date|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                           US	3653882	R3O9SG...|
|                                                                                                                                                                           US	14661224	RKH8B...|
|                             

In [14]:
# Count the number of reviews 
print(watch_df.count())

960872


## Drop duplicates and incomplete rows

In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

292
291
289


## Examine the schema

In [ ]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Employee ID: integer (nullable = true)
 |-- Email: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Hire Date: string (nullable = true)
 |-- Salaried: integer (nullable = true)
 |-- Vacation Hours: integer (nullable = true)
 |-- Sick Leave Hours: integer (nullable = true)
 |-- Encrypted Password: string (nullable = true)
 |-- Modified: string (nullable = true)



## Rename columns

In [ ]:
df1 = df.withColumnRenamed("Employee ID", "employee_id") \
        .withColumnRenamed("Email", "email") \
        .withColumnRenamed("Gender", "gender") \
        .withColumnRenamed("Hire Date", "hire_date") \
        .withColumnRenamed("DOB", "dob") \
        .withColumnRenamed("Encrypted Password", "password")
df1.show(5)

+---+-----------+-------+--------------------+--------+--------------+------+---------+--------+--------------+----------------+--------------------+------------+
|_c0|employee_id|  email|            Position|     dob|Marital Status|gender|hire_date|Salaried|Vacation Hours|Sick Leave Hours|            password|    Modified|
+---+-----------+-------+--------------------+--------+--------------+------+---------+--------+--------------+----------------+--------------------+------------+
| 23|  767955365|  mary0|Marketing Specialist| 9/13/62|             M|     F|  1/12/09|       0|            46|              43|90899830-127E-487...|6/30/14 0:00|
|142|  163347032|olinda0|Production Techni...|  4/4/70|             S|     F|   3/4/10|       0|             3|              21|6456CEC3-9B1E-4EB...|6/30/14 0:00|
|171|  746373306| david3|Production Techni...|  8/3/81|             S|     M|   1/2/09|       0|            32|              36|84C358D4-2493-452...|6/30/14 0:00|
|253|  407505660| lind

## Create a new DataFrame for employee info

In [ ]:
employee_personal_info = df1.select(["employee_id", "email", "gender", "hire_date", "dob"])
employee_personal_info.show(5)

+-----------+-------+------+---------+--------+
|employee_id|  email|gender|hire_date|     dob|
+-----------+-------+------+---------+--------+
|  767955365|  mary0|     F|  1/12/09| 9/13/62|
|  163347032|olinda0|     F|   3/4/10|  4/4/70|
|  746373306| david3|     M|   1/2/09|  8/3/81|
|  407505660| linda2|     F| 12/17/09|11/30/70|
|  658797903|  gigi0|     F|  1/16/09| 1/21/79|
+-----------+-------+------+---------+--------+
only showing top 5 rows



## Write DataFrame to RDS

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://mypostgreshwdb.cf9s0qdbvngm.us-east-1.rds.amazonaws.com/my_data_HW_db"
config = {"user":"root",
          "password": "colleen12",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

employee_personal_info.write.jdbc(url=jdbc_url, table='employee_personal_info', mode=mode, properties=config)

Py4JJavaError: ignored

## Create a new DataFrame for employee passwords

In [ ]:
employee_password = df1.select(["employee_id", "password"])
employee_password.show(5)

## Write DataFrame to RDS

In [ ]:
# Write DataFrame to table

employee_password.write.jdbc(url=jdbc_url, table='employee_password', mode=mode, properties=config)